In [1]:
# ## contains instituitons in US with info on location, level and classification

import subprocess
import sqlite3 as sqlite
import argparse
import os
import pandas as pd
import numpy as np 

# ## The link to helpers module as in proquest_to_db did not work
databasepath= "/mnt/ssd/AcademicGraph/"
db_file = f"{databasepath}AcademicGraph.sqlite"
con = sqlite.connect(database = db_file, isolation_level= None)
def analyze_db(con):
    """
    Run `analyze` commands on `con` according to sqlite recommendations.
    """
    print("Running ANALYZE... \n")
    cursor = con.cursor()
    cursor.execute("PRAGMA analysis_limit = 1000")
    cursor.execute("PRAGMA optimize")
    cursor.close()
    print("ANALYZE done")

# ## Read file
print("Reading file... \n")
datapath="/home/mona/mag_sample/data/us_institutions/"

institutions=pd.read_excel(datapath+"CCIHE2021-PublicData.xlsx", sheet_name="Data",
                           usecols = ["unitid", "name", "city", "stabbr", "iclevel", "basic2021"],
                           header=0,
                           dtype={"unitid": int, "iclevel":int, "basic2021":int})


print(institutions)

# ## Clean file: drop if unclassified institution: none
print("Cleaning... \n")
print("No institutions unspecified")

# ## Clean file: var as str okay? Or transform to int? Wasn't able to transform to int..

# ## Transfer to db 
print("Writing to db... \n")

db_input = {
    "institutions": {
            "df": institutions,
            "tb_name": "institutions",
            "schema": 
                """unitid INTEGER
                , name TEXT
                , city TEXT
                , stabbr TEXT
                , iclevel INTEGER
                , basic2021 INTEGER
                """,
            "create_idx": ["CREATE UNIQUE INDEX idx_instit ON institutions (institutions ASC)"]    
    }
}


for k, v in db_input.items():
    filename = databasepath + "tempfile.csv"
    v["df"].to_csv(filename, index = False, header = False) # sqlite does not recognize header rows
    tbl = v['tb_name']
    con.execute(f"DROP TABLE IF EXISTS {tbl}")
    con.execute(f"""CREATE TABLE {tbl}( {v['schema']} )""")
    subprocess.run(
        ["sqlite3", db_file,
        ".mode csv",
        f".import {filename} {tbl}"]
    )
    
   ## TODO: this is not working, what does it exactly do?
   # for command in v["create_idx"]:
   #     con.execute(command)
   # os.remove(filename)

# ## Analyze and finish 
analyze_db(con)

con.close()

print("Done.")

Reading file... 

      unitid                                               name          city  \
0     177834            A T Still University of Health Sciences    Kirksville   
1     134811  AI Miami International University of Art and D...         Miami   
2     429094       AOMA Graduate School of Integrative Medicine        Austin   
3     404994                                        ASA College      Brooklyn   
4     446127                               ATA Career Education   Spring Hill   
...      ...                                                ...           ...   
3934  493619     Young Americans College of the Performing Arts        Corona   
3935  141361                               Young Harris College  Young Harris   
3936  206695                        Youngstown State University    Youngstown   
3937  126119                                       Yuba College    Marysville   
3938  204255                                 Zane State College    Zanesville   

     stab